In [2]:
#%%
import natsort
import glob
import os
from os.path import join
import time
import re
import serial

ser = None

def sendData(data, port):
    ser = serial.Serial(('/dev/pts/' + port), 9600)
    data = str(data) + "\r\n"
    ser.write(data.encode())
    
# Define caminho dos arquivos.
mypath = "/media/ubuntu_storage/Projeto_Aplicado_XP/Dados/IMS/IMS/3rd_test/4th_test"

# Define lógica para fazer a listagem de todos os arquivos no diretorio.
files = glob.glob(join(mypath, "**/*"), recursive=True)
files = [f for f in files if os.path.isfile(f)]

# Faz com que a listagem dos arquivos esteja em ordem crescente, seguindo o padrão qual foram medidos os dados dos sensores.
files_sorted = natsort.natsorted(files,reverse=False)

# Realiza a leitura de linha a linha dos arquivos.
for y in range(len(files_sorted)):
    with open(files_sorted[y]) as f:
        lines = f.readlines()

        buffer = []  # Armazena as linhas até acumular 500
        max_lines = 500  # Limite de linhas para o envio em bloco

        for x in range(len(lines)):
            # Organiza os canais na medição linha a linha. Retirando o \t (tab) presente entre os canais e removendo o \n (nova linha)
            line_sorted = ''.join(lines[x])
            line_sorted = re.sub('\s+', ' ', line_sorted).split(' ')
            del line_sorted[-1]

            # Adiciona a linha processada ao buffer
            buffer.append({'Canal 1': float(line_sorted[0]), 'Canal 2': float(line_sorted[1]), 'Canal 3': float(line_sorted[2]), 'Canal 4': float(line_sorted[3])})

            # Quando acumular 500 linhas, envia os dados e limpa o buffer
            if len(buffer) >= max_lines:
                # Envia os dados acumulados (neste caso, envia apenas o Canal 3, mas pode adaptar para enviar mais)
                for data in buffer:
                    sendData(data['Canal 3'], '12')

                buffer = []  # Limpa o buffer após o envio
                time.sleep(5)  # Pausa de 1 segundo antes de continuar processando mais linhas

        # Verifica se restaram dados no buffer e os envia
        if buffer:
            for data in buffer:
                sendData(data['Canal 3'], '12')
            
            


SerialException: [Errno 2] could not open port /dev/pts/11: [Errno 2] No such file or directory: '/dev/pts/11'

In [6]:
#%%
import natsort
import glob
import os
from os.path import join
import time
import re
import serial

ser = None

def sendData(data, port):
    """
    Função para enviar os dados via porta serial.
    
    data: Dado a ser enviado
    port: Porta serial utilizada para o envio
    """
    ser = serial.Serial(('/dev/pts/' + port), 9600)
    data += "\r\n"
    ser.write(data.encode())
    ser.close()  # Fecha a porta após enviar os dados

# Define caminho dos arquivos (do terceiro experimento).
mypath = "/media/ubuntu_storage/Projeto_Aplicado_XP/Dados/IMS/IMS/3rd_test/4th_test"

# Define lógica para fazer a listagem de todos os arquivos no diretório.
files = glob.glob(join(mypath, "**/*"), recursive=True)
files = [f for f in files if os.path.isfile(f)]

# Faz com que a listagem dos arquivos esteja em ordem crescente, seguindo o padrão qual foram medidos os dados dos sensores.
files_sorted = natsort.natsorted(files, reverse=False)

# Regex para organizar os canais na medição linha a linha.
channel_regex = re.compile(r'\s+')

# Parâmetros de aquisição do experimento
sampling_rate = 20000  # 20 kHz, ou seja, 20.000 amostras por segundo
points_per_file = 20480  # Cada arquivo tem 20.480 pontos (1 segundo de dados)
interval_between_files = 600  # Intervalo de 10 minutos entre os arquivos (600 segundos)

# Loop para enviar os dados de cada arquivo do rolamento 3
try:
    while True:
        # Itera sobre os arquivos no diretório
        for filename in files_sorted:
            if file_regex.match(filename):  # Verifica se o arquivo corresponde ao padrão esperado
                file_path = os.path.join(mypath, filename)

                # Abre e lê os dados do arquivo
                with open(file_path) as f:
                    lines = f.readlines()

                    # Para cada linha (20.480 pontos por arquivo)
                    for x in range(len(lines)):
                        # Organiza os canais na medição linha a linha. 
                        line_sorted = ''.join(lines[x])
                        line_sorted = re.sub('\s+', ' ', line_sorted).split(' ')
                        del line_sorted[-1]  # Remove o último item, que pode ser um caractere vazio
                        
                        # Envia o valor do Canal 3 (rolamento 3, falha no terceiro experimento)
                        sendData(line_sorted[2], '11')  # Enviando via porta /dev/pts/11

                        # Aguarda o próximo valor com base na frequência de amostragem (1/20.000 segundos)
                        time.sleep(1 / sampling_rate)

                # Após o envio de todos os dados do arquivo, espera 10 minutos antes de passar para o próximo arquivo
                time.sleep(interval_between_files)
except KeyboardInterrupt:
    print("Interrompido pelo usuário")
finally:
    if ser:
        ser.close()

Interrompido pelo usuário
